In [ ]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader

In [ ]:
#

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])


train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=False, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#

model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
num_classes = 10
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)

criterion = nn.CrossEntropyLoss()

optimizer = optim.AdamW(model.fc.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

#

num_epochs = 5

for epoch in range(num_epochs):
    print(f"Epoch # {epoch + 1}------------------------------------------")
    
    loss_per_epoch = 0.0
    
    model.train()
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)#.float()
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        _, preds = torch.max(outputs, 1)
        
        preds = preds.float()
        
        #loss = criterion(preds, labels)
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        loss_per_epoch += loss.item() * images.size(0)
        
    loss_per_epoch /= len(train_loader.dataset) ####
    
    print(f"Epoch: {epoch + 1} / {num_epochs}, Loss in this epoch: {loss_per_epoch}")